# **LIMPEZA TABELA BRASILEIRÃO 2003 - 2021**

Aqui vamos realizar a limpeza dos dados, padronizando e ajustando valores, além de remover colunas não úteis.

### **ENTENDENDO O CABEÇALHO**

Abaixo consta a explicação do cabeçalho dos dados que vamos tratar.

- **Pos.**: Posição da equipe.
- **Equipes**: Equipes que participaram do campeonato.
- **P**: Pontuação.
- **J**: Total de Jogos.
- **V**: Vitórias.
- **E**: Empates.
- **D**: Derrotas.
- **GP**: Gols Pró (gols feitos).
- **GC**: Gols contras (Gols sofridos).
- **SG**: Saldo de Gols.
- **%**: Porcentagem da equipe em toda a competição.
- **Classificação ou rebaixamento**: Situação final da equipe.
- **ano**: Ano que foi realizado o campeonato.

### **IMPORTANDO LIBS E CARREGANDO O ARQUIVO**

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df_tbra = pd.read_json('tabelasBrasileirao2003_2021.json')

## **REMOVENDO COLUNAS NÃO ÚTEIS**

A coluna **_M_** representa a movimentação do time na última rodada, uma informação que para nossa análise não será útil.
Abaixo temos a remoção do item.

In [4]:
df_tbra.drop(columns='M', inplace=True)
df_tbra.head()

,Pos.,Equipes,P,J,V,E,D,GP,GC,SG,%,Classificação ou rebaixamento,ano
0,1,Cruzeiro,100,46,31,7,8,102,47,+55,72,Fase de grupos da Copa Libertadores de 2004,2003
1,2,Santos,87,46,25,12,9,93,60,+33,63,Fase de grupos da Copa Libertadores de 2004,2003
2,3,São Paulo,78,46,22,12,12,81,67,+14,56,Fase de grupos da Copa Libertadores de 2004,2003
3,4,São Caetano,742,46,19,14,13,53,37,+16,53,Fase de grupos da Copa Libertadores de 2004,2003
4,5,Coritiba,73,46,21,10,15,67,58,+9,52,Fase de grupos da Copa Libertadores de 2004,2003


## **ANÁLISE DAS INFORMAÇÕES DA BASE**

Abaixo vamos analisar as informações das colunas, verificando seu tipo e se precisa realizar ajuste no mesmo.

In [5]:
df_tbra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Pos.                           390 non-null    int64 
 1   Equipes                        390 non-null    object
 2   P                              390 non-null    int64 
 3   J                              390 non-null    int64 
 4   V                              390 non-null    int64 
 5   E                              390 non-null    int64 
 6   D                              390 non-null    int64 
 7   GP                             390 non-null    int64 
 8   GC                             390 non-null    int64 
 9   SG                             390 non-null    object
 10  %                              390 non-null    int64 
 11  Classificação ou rebaixamento  390 non-null    object
 12  ano                            390 non-null    int64 
dtypes: in

Notamos que o tipo da coluna **_SG_** (_saldo de gols_) consta como object (tipo string). 
Como esta coluna represanta o total de gols no resultado da conta de **_GP_** - **_GC_**, vamos realizar o cálculo.

In [6]:
df_tbra['SG'] = df_tbra['GP'] - df_tbra['GC']
df_tbra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Pos.                           390 non-null    int64 
 1   Equipes                        390 non-null    object
 2   P                              390 non-null    int64 
 3   J                              390 non-null    int64 
 4   V                              390 non-null    int64 
 5   E                              390 non-null    int64 
 6   D                              390 non-null    int64 
 7   GP                             390 non-null    int64 
 8   GC                             390 non-null    int64 
 9   SG                             390 non-null    int64 
 10  %                              390 non-null    int64 
 11  Classificação ou rebaixamento  390 non-null    object
 12  ano                            390 non-null    int64 
dtypes: in

Após este ajuste, vamos verificar as variáveis do tipo object, para analise dos valores, se precisamos realizar algum ajuste ou padronização.
Vamos começar com a variável **_Equipes_** depois **_Classificação ou rebaixamento_**

In [7]:
df_tbra['Equipes'].sort_values().unique()

array(['América Mineiro', 'América de Natal', 'Athletico Paranaense',
       'Atlético Goianiense', 'Atlético Mineiro', 'Atlético Paranaense',
       'Avaí', 'Bahia', 'Botafogo', 'Brasiliense', 'CSA', 'Ceará',
       'Chapecoense', 'Corinthians', 'Coritiba', 'Criciúma', 'Cruzeiro',
       'Cuiabá', 'Figueirense', 'Flamengo', 'Fluminense', 'Fortaleza',
       'Goiás', 'Grêmio', 'Grêmio Barueri', 'Grêmio Prudente', 'Guarani',
       'Internacional', 'Ipatinga', 'Joinville', 'Juventude', 'Náutico',
       'Palmeiras', 'Palmeiras1', 'Paraná', 'Paysandu', 'Ponte Preta',
       'Portuguesa', 'Red Bull Bragantino', 'Santa Cruz', 'Santo André',
       'Santos', 'Sport', 'Sport[nota 4]', 'São Caetano', 'São Paulo',
       'Vasco da Gama', 'Vitória'], dtype=object)

Notamos que precisa ser realizado alguns ajustes de valores nos nomes que são itens que de formatação que veio quando realizamos o _web scraping_, como nas equipes **_Palmeiras1_** e **_Sport\[nota 4\]_**.

Já no caso do **_Athlético Paranaense_**, houve a mudança na nomenclatura em 2018 (aproximadamente), para que possamos realizar futuras análises, vamos deixar o formato atual para todas as tabelas.

Vamos aproveitar e verificar a coluna de **_Classificação ou rebaixamento_**.

In [8]:
df_tbra['Classificação ou rebaixamento'].sort_values().unique()

array(['Copa Sul-Americana de 20043', 'Copa Sul-Americana de 2005',
       'Copa Sul-Americana de 2006', 'Copa Sul-Americana de 2007',
       'Copa Sul-Americana de 2008', 'Copa Sul-Americana de 2009',
       'Copa Sul-Americana de 20092', 'Copa Sul-Americana de 2010',
       'Copa Sul-Americana de 2011', 'Copa Sul-Americana de 2012',
       'Copa Sul-Americana de 2017', 'Copa Sul-Americana de 2018',
       'Copa Sul-Americana de 2019', 'Copa Sul-Americana de 2020',
       'Fase de grupos da Copa Libertadores de 2004',
       'Fase de grupos da Copa Libertadores de 2017',
       'Fase de grupos da Copa Libertadores de 20171',
       'Fase de grupos da Copa Libertadores de 2018[nota 3]',
       'Fase de grupos da Copa Libertadores de 2019',
       'Fase de grupos da Copa Libertadores de 2019[nota 3]',
       'Fase de grupos da Copa Libertadores de 2020[nota 3]',
       'Fase de grupos da Copa Libertadores de 2021',
       'Fase de grupos da Copa Libertadores de 2021[nota 5]',
       'Fa

As informações que contém nele, para nossa análise, precisamos saber: Quem foi para a fase de grupos da libertadores e sul-americana, segunda fase da libertadores e rebaixamento; as informações de ano não nos sera relevante nesta análise, com isso, vamos remover os valores que não são relevantes destas frases.

Abaixo vamos unir todas a padronizações que precisamos e realizar os replaces necessários.

In [9]:
df_tbra = (df_tbra
              .replace('Palmeiras1', 'Palmeiras')
              .replace('Atlético Paranaense', 'Athletico Paranaense')
              .replace('null', np.NaN)
              .replace(to_replace=r'\[(n|N)ota \d\]', value='', regex=True)
              .replace(to_replace=r'( de \d+)$', value='', regex=True)
              .replace(to_replace=r'(Fase de grupos|((Primeira|Segunda) fase)) da ', value='', regex=True)
            
          )

In [10]:
df_tbra['Equipes'].sort_values().unique()

array(['América Mineiro', 'América de Natal', 'Athletico Paranaense',
       'Atlético Goianiense', 'Atlético Mineiro', 'Avaí', 'Bahia',
       'Botafogo', 'Brasiliense', 'CSA', 'Ceará', 'Chapecoense',
       'Corinthians', 'Coritiba', 'Criciúma', 'Cruzeiro', 'Cuiabá',
       'Figueirense', 'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás',
       'Grêmio', 'Grêmio Barueri', 'Grêmio Prudente', 'Guarani',
       'Internacional', 'Ipatinga', 'Joinville', 'Juventude', 'Náutico',
       'Palmeiras', 'Paraná', 'Paysandu', 'Ponte Preta', 'Portuguesa',
       'Red Bull Bragantino', 'Santa Cruz', 'Santo André', 'Santos',
       'Sport', 'São Caetano', 'São Paulo', 'Vasco da Gama', 'Vitória'],
      dtype=object)

In [11]:
df_tbra['Classificação ou rebaixamento'].sort_values().unique()

array(['Copa Libertadores', 'Copa Sul-Americana', 'Rebaixados à Série B',
       nan], dtype=object)

Feito os ajustes nos textos, vamos agora verificar os itens numéricos.

In [12]:
df_tbra.describe()

,Pos.,P,J,V,E,D,GP,GC,SG,%,ano
count,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000
mean,10.802564,79.176923,39.210256,14.438462,10.328205,14.443590,50.602564,50.617949,-0.015385,45.412821,2011.789744
std,5.999957,262.884723,2.708057,4.352118,2.666555,4.384554,13.155917,12.568995,16.517803,10.330345,5.568170
min,1.000000,15.000000,38.000000,1.000000,3.000000,4.000000,18.000000,19.000000,-57.000000,13.000000,2003.000000
25%,6.000000,46.000000,38.000000,11.000000,8.000000,11.000000,42.000000,42.000000,-10.000000,39.000000,2007.000000
50%,11.000000,53.000000,38.000000,14.000000,10.000000,14.000000,50.000000,49.000000,-1.000000,45.000000,2012.000000
75%,16.000000,62.000000,38.000000,17.000000,12.000000,17.000000,59.000000,58.000000,10.750000,52.000000,2017.000000
max,24.000000,5012.000000,46.000000,31.000000,18.000000,29.000000,103.000000,92.000000,55.000000,79.000000,2021.000000


Podemos notar que a quantidade total esta correta, todas batem, isso indica que nenhuma tem valores **_NaN_**.
Porém, quando observamos valores como o variável **_P_**, vemos que existem uma falha, não é possível ter pontuação máxima de 5012, temos que verificar o que esta havendo nesta variável.

### **VERIFICANDO A VARIÁVEL _P_**

In [13]:
def ajustePontuacoes(pontuacao):
    pt_str = str(pontuacao['P'])
    if pontuacao['P'] > 1000:
        pt_str = re.sub(r'(\d\d)$', '', pt_str)
    else:
        pt_str = re.sub(r'(\d)$', '', pt_str)
    pontuacao['P'] = int(pt_str)
    
    return pontuacao
    
df_tbra[df_tbra['P']>100] = df_tbra[df_tbra['P']>100].apply(ajustePontuacoes, axis=1)

In [14]:
df_tbra.describe()

,Pos.,P,J,V,E,D,GP,GC,SG,%,ano
count,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000
mean,10.802564,53.546154,39.210256,14.438462,10.328205,14.443590,50.602564,50.617949,-0.015385,45.412821,2011.789744
std,5.999957,12.686113,2.708057,4.352118,2.666555,4.384554,13.155917,12.568995,16.517803,10.330345,5.568170
min,1.000000,15.000000,38.000000,1.000000,3.000000,4.000000,18.000000,19.000000,-57.000000,13.000000,2003.000000
25%,6.000000,45.000000,38.000000,11.000000,8.000000,11.000000,42.000000,42.000000,-10.000000,39.000000,2007.000000
50%,11.000000,53.000000,38.000000,14.000000,10.000000,14.000000,50.000000,49.000000,-1.000000,45.000000,2012.000000
75%,16.000000,61.000000,38.000000,17.000000,12.000000,17.000000,59.000000,58.000000,10.750000,52.000000,2017.000000
max,24.000000,100.000000,46.000000,31.000000,18.000000,29.000000,103.000000,92.000000,55.000000,79.000000,2021.000000


## **ADICIONANDO VARIÁVEL**

Será adicionado a variável de **FD** (Federação), onde conterá a Unidade Federativa (Estado) da Equipe.

In [15]:
fd_dict = {
            'MG':['Cruzeiro', 'Atlético Mineiro','América Mineiro','Ipatinga'],
            'SP':['Santos', 'São Paulo', 'São Caetano','Guarani', 'Palmeiras', 'Corinthians', 'Ponte Preta', 'Portuguesa','Grêmio Barueri', 'Santo André','Grêmio Prudente','Red Bull Bragantino'],
            'RS':['Internacional', 'Grêmio','Juventude'], 
            'SC':['Joinville','Figueirense','Criciúma','Avaí','Chapecoense'],
            'RJ':['Flamengo', 'Fluminense', 'Vasco da Gama', 'Botafogo'],
            'GO':['Goiás', 'Atlético Goianiense'],
            'PR':['Paraná','Athletico Paranaense', 'Coritiba'],
            'CE':['Fortaleza','Ceará'],
            'PA':['Paysandu'],
            'BA':['Bahia', 'Vitória'],
            'DF':['Brasiliense'], 
            'PE': ['Santa Cruz', 'Sport', 'Náutico'],
            'RN':['América de Natal'], 
            'MT':['Cuiabá'],
            'AL':['CSA']
        }



for fd in fd_dict:
    df_tbra.loc[df_tbra['Equipes'].isin(fd_dict[fd]),'fd'] = fd
df_tbra

,Pos.,Equipes,P,J,V,E,D,GP,GC,SG,%,Classificação ou rebaixamento,ano,fd
0,1,Cruzeiro,100,46,31,7,8,102,47,55,72,Copa Libertadores,2003,MG
1,2,Santos,87,46,25,12,9,93,60,33,63,Copa Libertadores,2003,SP
2,3,São Paulo,78,46,22,12,12,81,67,14,56,Copa Libertadores,2003,SP
3,4,São Caetano,74,46,19,14,13,53,37,16,53,Copa Libertadores,2003,SP
4,5,Coritiba,73,46,21,10,15,67,58,9,52,Copa Libertadores,2003,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,16,Juventude,46,38,11,13,14,36,44,-8,40,NaN,2021,RS
386,17,Grêmio,43,38,12,7,19,44,51,-7,38,Rebaixados à Série B,2021,RS
387,18,Bahia,43,38,11,10,17,42,51,-9,38,Rebaixados à Série B,2021,BA
388,19,Sport,38,38,9,11,18,24,37,-13,33,Rebaixados à Série B,2021,PE


## **SALVANDO RESULTADO DA MINERAÇÃO**

In [16]:
df_tbra.to_csv('tabelasBrasileirao.csv', index=False)

## **RESULTADO FINAL DOS CABEÇALHOS**

- **Pos.**: Posição da equipe.
- **Equipes**: Equipes que participaram do campeonato.
- **P**: Pontuação.
- **J**: Total de Jogos.
- **V**: Vitórias.
- **E**: Empates.
- **D**: Derrotas.
- **GP**: Gols Pró (gols feitos).
- **GC**: Gols contras (Gols sofridos).
- **SG**: Saldo de Gols.
- **%**: Porcentagem da equipe em toda a competição.
- **Classificação ou rebaixamento**: Situação final da equipe.
- **ano**: Ano que foi realizado o campeonato.
- **fd**: Unidade Federativa (Estado) da equipe.